<a href="https://colab.research.google.com/github/jjhonxpx/confeitaria_projeto01/blob/main/SDV/SDV_teste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instala a biblioteca SDV
!pip install sdv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.6/196.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.3/198.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 5.7 MB/s eta 0:00:00


In [ ]:
# Importa a biblioteca pandas, usada para manipulação de dados tabulares
import pandas as pd

# Importa o sintetizador GaussianCopula do SDV, modelo estatístico para gerar dados sintéticos a partir de uma única tabela
from sdv.single_table import GaussianCopulaSynthesizer

# Importa a classe para criar ou detectar automaticamente o metadado da sua tabela
from sdv.metadata import SingleTableMetadata

# Importa a classe FixedCombinations, que impede o SDV de criar combinações entre colunas que não existem nos dados originais
from sdv.cag import FixedCombinations


In [ ]:
# Cria um DataFrame com dados de produtos de uma confeitaria
confeitaria_df = pd.DataFrame({
    'Produto': ['Brigadeiro', 'Coxinha', 'Quindim', 'Empada', 'Bolo de Cenoura', 'Pastel', 'Beijinho', 'Pão de Queijo'],
    'Tipo': ['Doce', 'Salgado', 'Doce', 'Salgado', 'Doce', 'Salgado', 'Doce', 'Salgado'],
    'Preço (R$)': [3.00, 5.00, 4.50, 5.50, 8.00, 6.00, 3.50, 4.00],
    'Custo (R$)': [1.20, 2.00, 1.80, 2.50, 4.50, 2.20, 1.40, 1.80],
    'Quantidade Vendida': [120, 90, 50, 60, 40, 100, 80, 110]
})

confeitaria_df

,Produto,Tipo,Preço (R$),Custo (R$),Quantidade Vendida
0,Brigadeiro,Doce,3.0,1.2,120
1,Coxinha,Salgado,5.0,2.0,90
2,Quindim,Doce,4.5,1.8,50
3,Empada,Salgado,5.5,2.5,60
4,Bolo de Cenoura,Doce,8.0,4.5,40
5,Pastel,Salgado,6.0,2.2,100
6,Beijinho,Doce,3.5,1.4,80
7,Pão de Queijo,Salgado,4.0,1.8,110


In [ ]:
# Colunas que devem manter apenas as combinações existentes nos dados reais
constraint = FixedCombinations(
    column_names=['Produto', 'Tipo']
)

In [ ]:
# Gera o metadado automaticamente
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(confeitaria_df)
print('Metadado detectado:', metadata.to_dict()) # Veja o tipo de dado de cada coluna

Metadado detectado: {'METADATA_SPEC_VERSION': 'SINGLE_TABLE_V1', 'columns': {'Produto': {'sdtype': 'id'}, 'Tipo': {'sdtype': 'categorical'}, 'Preço (R$)': {'sdtype': 'numerical'}, 'Custo (R$)': {'sdtype': 'numerical'}, 'Quantidade Vendida': {'sdtype': 'numerical'}}, 'primary_key': 'Produto'}


In [ ]:
# Não definir nenhuma chave primária
metadata.primary_key = None

In [ ]:
# Garante que a coluna 'Produto' será categórica
metadata.update_column(column_name='Produto', sdtype='categorical')

In [ ]:
# Cria o sintetizador
synthesizer = GaussianCopulaSynthesizer(metadata)

# Adiciona restrições ao sintetizador usando add_constraints
synthesizer.add_constraints([constraint])  # Lista de constraints

/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:168: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:134: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [ ]:
# Ajustar o modelo ao seu conjunto real
synthesizer.fit(confeitaria_df)

In [ ]:
# Gerar novas linhas sintéticas
sintetico = synthesizer.sample(num_rows=100)
print(sintetico.head())

           Produto     Tipo  Preço (R$)  Custo (R$)  Quantidade Vendida
0         Beijinho     Doce         6.2         3.1                  71
1         Beijinho     Doce         5.7         2.7                  63
2           Empada  Salgado         4.0         1.4                 108
3           Pastel  Salgado         5.8         2.6                  55
4  Bolo de Cenoura     Doce         6.1         2.6                  68


In [ ]:
pd.options.display.max_rows = None  # Permite mostrar todas as linhas

sintetico

,Produto,Tipo,Preço (R$),Custo (R$),Quantidade Vendida
0,Beijinho,Doce,6.2,3.1,71
1,Beijinho,Doce,5.7,2.7,63
2,Empada,Salgado,4.0,1.4,108
3,Pastel,Salgado,5.8,2.6,55
4,Bolo de Cenoura,Doce,6.1,2.6,68
5,Empada,Salgado,3.9,1.4,68
6,Beijinho,Doce,7.7,4.5,50
7,Quindim,Doce,5.0,1.7,59
8,Empada,Salgado,7.1,3.6,74
9,Coxinha,Salgado,4.6,1.5,111


In [ ]:
# Exportar planilha sintética em excel e csv
sintetico.to_csv('dados_sinteticos_sdv.csv', index=False)
sintetico.to_excel('dados_sinteticos_sdv.xlsx', index=False)